In [ ]:
#Importing  libraries 
import pandas as pd 
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import os 
import glob as gb
import cv2
import keras 

In [ ]:
train_path = "../input/esophageal-cancer/ec/train/"
test_path = "../input/esophageal-cancer/ec/test/"
predict_path = "../input/esophageal-cancer/ec/predict/"

In [ ]:
for folder in os.listdir(train_path):
    files = gb.glob(pathname=str(train_path + folder + '/*.PNG'))
    print(f'For training data , found {len(files)} in folder {folder}')

In [ ]:
for folder in os.listdir(test_path):
    files = gb.glob(pathname=str(test_path + folder + '/*.PNG'))
    print(f'For testing data , found {len(files)} in folder {folder}')

In [ ]:
files = gb.glob(pathname= str(predict_path +'/*.PNG'))
print(f'For Prediction data , found {len(files)}')

In [ ]:
code = {'Adenocarcinoma-stage-I' : 0 , 'Adenocarcinoma-stage-IIB' : 1 , 'Adenocarcinoma-Stage-III':2 , 'Adenocarcinoma-stage-IIIA' : 3 , 'Adenocarcinoma-stage-IV' : 4 , 'Adenocarcinoma-stage-IVA' : 5 , 'Squamous-cell-carcinoma-stage-IA' : 6 , 'Squamous-cell-carcinoma-stage-IB' : 7 , 'Squamous-cell-carcinoma-stage-IIA' : 8 }

def title(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x


In [ ]:
#Getting the sizes of all the images of the training set
size = []
for folder in os.listdir(train_path):
    files = gb.glob(pathname = str(train_path + folder + '/*.PNG'))
    for file in files:
        img = plt.imread(file)
        size.append(img.shape)
pd.Series(size).value_counts()

In [ ]:
#Getting the sizes of all the images of the test set 
size = []
for folder in os.listdir(test_path):
    files = gb.glob(pathname = str(test_path + folder + '/*.PNG'))
    for file in files:
        img = plt.imread(file)
        size.append(img.shape)
pd.Series(size).value_counts()

In [ ]:
s = 350

In [ ]:
x_train = []
y_train = []
for folder in os.listdir(train_path):
    files = gb.glob(pathname=str(train_path + folder + '/*.PNG' ))
    for file in files:
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        x_train.append(list(image_array))
        y_train.append(code[folder])

In [ ]:
print(f'well , now we have a {len(x_train)} items in x_train')

In [ ]:
#getting&plotting the images
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_train),36))):
    plt.subplot(6,6,n+1)
    plt.imshow(x_train[i])
    plt.axis('off')
    plt.title(title(y_train[i]))


In [ ]:
x_test = []
y_test = []
for folder in os.listdir(test_path):
    files = gb.glob(pathname=str(train_path + folder + '/*.PNG' ))
    for file in files:
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        x_test.append(list(image_array))
        y_test.append(code[folder])

In [ ]:
#getting&plotting the images for testing set images
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_test),36))):
    plt.subplot(6,6,n+1)
    plt.imshow(x_train[i])
    plt.axis('off')
    plt.title(title(y_test[i]))


In [ ]:
#Getting and preparing the predinction data images 
x_pred = []
files = gb.glob(pathname= str(predict_path + '/*.PNG'))
for file in files: 
    image = cv2.imread(file)
    image_array = cv2.resize(image , (s,s))
    x_pred.append(list(image_array))

In [ ]:
print(f'For the predictiond data we found {len(x_pred)} images')

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(x_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(x_pred[i])    
    plt.axis('off')

In [ ]:
#performing the data augmentaion 
data_augmentation = tf.keras.Sequential([
     tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
     tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
predict_path = tf.expand_dims(predict_path , 0) 

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(s,s,3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

**BUILDING THE CNN MODEL FOR PREDICTION**

In [ ]:
image_batch, label_batch = next(iter(predict_path))
feature_batch = base_model(image_batch) 
print(feature_batch.shape) 

In [ ]:
#adding the classification head 
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_pred = np.array(x_pred)

print(f'x_train shape is {x_train.shape}')
print(f'x_train shape is {y_train.shape}')
print(f'x_train shape is {x_test.shape}')
print(f'x_train shape is {y_test.shape}')
print(f'x_train shape is {x_pred.shape}')

In [ ]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)),
        keras.layers.Conv2D(150,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(2,2),
        keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(100,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(120,activation='relu') ,    
        keras.layers.Dense(100,activation='relu') ,    
        keras.layers.Dense(80,activation='relu') ,        
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(9,activation='softmax') ,    
        ])

**Compiling the model using adams optimizer & sparse catergotical crossentrop loss**

In [ ]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print('Model details')
print(KerasModel.summary())

**Training the model **

In [ ]:
#ThisModel = KerasModel.fit(x_train, y_train, epochs=50,batch_size=16,verbose=1) 